In [16]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
import os
import numpy as np
from IPython.display import display, HTML

%matplotlib inline

In [18]:
#read calendar events from google
#requirement: pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle

scopes = ['https://www.googleapis.com/auth/calendar']
flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes=scopes)
credentials = flow.run_console()

#save credentials
pickle.dump(credentials, open("token.pkl", "wb")) 
credentials = pickle.load(open("token.pkl", "rb"))

service = build('calendar', 'v3', credentials=credentials)
result = service.calendarList().list().execute()

#specify desired calendar to access
your_calendar_id = 'YOUR_CALENDAR_ID'

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=488185422413-7tpmaorfd7ogrd33991srev9qdpkd1sl.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=igFiBGKS5DOf5g2ZrzvhbxEwR6vFYk&prompt=consent&access_type=offline
Enter the authorization code: 4/vQEPz0glBzqVx5pSSFxJB2lafbte76NTXEyRf2ThCJ9JpZqzyXFGv6Q


In [ ]:
# Call the Calendar API
from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse as parse_date
import pdb

#deal with timezone issue for comparison
import pytz
utc=pytz.UTC

#specify max date of calendar events you would like to access (days in future)
days_in_future = 365

now = datetime.utcnow()
date_final = (datetime.utcnow() +  timedelta(days=days_in_future)).isoformat() + 'Z'

#start_date - earliest date of calendar events needed
start_date_str = '09/01/19'
start_date = datetime.strptime(start_date_str, '%m/%d/%y').isoformat() + 'Z'

events_result = service.events().list(calendarId=your_calendar_id, 
                                    timeMin=start_date, 
                                    timeMax=date_final,
                                    maxResults=1000,
                                    singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])
if not events:
    print('no events found!')

#make empty list to insert with event dictionaries
#calculate duration of event
#logic to dictate if event is in past or future (paid or booked)
event_list = list()
for event in events:
    start = parse_date(event['start'].get('dateTime'))
    end = parse_date(event['end'].get('dateTime'))
    event_duration = end - start
    event_duration_minutes = event_duration.seconds // 60
    if end < utc.localize(now):
        status = 'PAID'
    else:
        status = 'BOOKED'
    cal_entry_dict={'date': start.strftime('%Y-%m-%d'), 
                    'client': event['summary'], 
                    'minutes':event_duration_minutes,
                   'status' : status}
    event_list.append(cal_entry_dict)

#Convert to Pandas Dataframe
classes_df = pd.DataFrame(event_list) 

